# 🦠 Visualização de Dados da COVID-19

## 🏁 Introdução
Utilização de dados do **AWS COVID-19 data lake** para analisar informações precisas sobre a disseminação e as características do coronavírus (SARS-CoV-2). 

Com esses dados, há a apresentação de diversas visualizações, explorando várias correlações entre eles. Destacam-se o **gráfico de barras**, que compara a disponibilidade de leitos hospitalares entre diferentes regiões, e o **mapa interativo da linha do tempo**, que mostra a evolução de mortes mundialmente. 

O objetivo principal é compreender as informações obtidas no data lake e apresentar as visualizações mais eficazes para gerar **insights pertinentes**. 

## 📚Importação das bibliotecas

In [1]:
import numpy as np
import pandas as pd
from io import StringIO
import plotly.express as px
from us_elections_president.filtered_data import elections_president_2020
from config.aws_config import session, REGION_NAME, BUCKET_NAME, BUCKET_KEY_HOSPITAL_BEDS, BUCKET_KEY_CASES_AND_DEATHS, BUCKET_KEY_GLOBAL_DEATHS, BUCKET_KEYS_VACCINATED_PER_HUNDRED, BUCKET_KEY_DOSES_PFIZER

## 🔒 Recuperando credenciais da AWS e configurando a conexão S3

In [2]:
def run_s3(BUCKET_KEY):
    s3 = session.client('s3', region_name=REGION_NAME)
    response = s3.get_object(Bucket=BUCKET_NAME, Key=BUCKET_KEY)
    data = response['Body'].read().decode('utf-8')
    data_frame = pd.read_csv(StringIO(data))
    
    return data_frame    

## 👓Análises e Visualizações
### 1. Estados Americanos e camas disponíveis
Relação entre quantidade de leitos disponívveis em cada Estado Americano para auxílio no tratamento do coronavírus.

In [3]:
data_frame = run_s3(BUCKET_KEY_HOSPITAL_BEDS)

fig = px.bar(data_frame,
            x='hq_state', 
            y='total_staffed_beds', 
            color='hq_state', 
            labels={'hq_state': 'Estado', 'total_staffed_beds': 'Total de camas'},
            title='Estados Americanos e camas disponíveis')

fig.show()

### 2. Estados Americanos: Casos e Mortes
Linha do tempo pelos mêses de 2020 apresentando a evolução dos casos e mortes de americanos.

In [4]:
data_frame = run_s3(BUCKET_KEY_CASES_AND_DEATHS)

fig = px.scatter(data_frame,
                x='total_cases',
                y='total_deaths',
                animation_frame='month',
                animation_group='state_name',
                color='state_name',
                hover_name='state_name',
                size='total_cases',
                log_x=True,
                size_max=55,
                range_y=[min(data_frame['total_deaths']), max(data_frame['total_deaths'])],
                labels={'state_name':'Estado', 'month':'Mês', 'total_deaths':'Mortes', 'total_cases':'Casos'},
                title='Evolução Mensal de Casos e Mortes por COVID-19 por Estado Americano')
  
fig.show()

### 3. Mortes mundialmente
Linha do tempo pelos mêses de 2020 retratando mortes ao redor do mundo causadas pelo COVID-19.

In [5]:
data_frame = run_s3(BUCKET_KEY_GLOBAL_DEATHS)
data_frame['log_scaled_deaths'] = np.log10(data_frame['deaths_on_last_day'] + 1) * 5

fig = px.scatter_geo(data_frame,
                    locations='country_name',
                    locationmode='country names',
                    template='seaborn',
                    color='country_name', 
                    size='log_scaled_deaths',  
                    hover_data={'deaths_on_last_day': True, 'log_scaled_deaths': False}, 
                    animation_frame='month', 
                    projection="natural earth",
                    labels={'country_name':'País', 'month':'Mês', 'deaths_on_last_day':'Total de mortes no fim do mês'},
                    title="Evolução Mensal de Mortes por COVID-19 mundialmente",
                    size_max=20) 
    
fig.show()

### 4. Estados Americanos e a média de vacinas por 100 habitantes 
#### Mesclado com as maiores quantidades de votos para partidos eleitorais nas eleições de 2020
10 Maiores e Menores médias de pessoas totalmente vacinadas nos Estados Americanos, mesclada com dados sobre os partidos mais votados nesses estados nas eleições preidenciais de 2020.

In [6]:
data_frame = run_s3(BUCKET_KEYS_VACCINATED_PER_HUNDRED)
data_frame.rename(columns={'location':'Estado'}, inplace=True)

most_votes_by_party_df = elections_president_2020()
combined_elections_president_2020_with_averange_vaccinated_per_hundred_df = pd.merge(data_frame, most_votes_by_party_df, on='Estado', how='left')
combined_elections_president_2020_with_averange_vaccinated_per_hundred_df['party_simplified'] = combined_elections_president_2020_with_averange_vaccinated_per_hundred_df['party_simplified'].fillna('NO DATA')

party_colors = {
    'REPUBLICAN': '#FF3333',
    'DEMOCRAT': '#0033FF',
    'LIBERTARIAN': '#FFFF33',
    'NO DATA': '#CCCCCC'
}
fig = px.bar_polar(combined_elections_president_2020_with_averange_vaccinated_per_hundred_df, 
                    r='average_fully_vaccinated_per_hundred', 
                    theta='Estado', 
                    color='party_simplified', 
                    color_discrete_map=party_colors,
                    hover_data=['party_simplified', 'candidatevotes'],
                    template='plotly_white',
                    labels={'average_fully_vaccinated_per_hundred': 'Média de pessoas totalmente vacinadas por 100 habitantes'},
                    title='As 10 Menores e Maiores Médias de Pessoas Totalmente Vacinadas por 100 Habitantes em Estados Americanos no mês 12/2022') 
 
fig.add_annotation(
        x=0, y=1.16, 
        xref="paper", yref="paper",
        showarrow=False,
        text='Coloridas pelo Partido Eleitoral que soma o maior número nas Eleições Presidenciais de 2020',
        font=dict(size=16, color="black"),
        align="center"
)
    
fig.update_layout(polar=dict(
                    bgcolor='rgba(255, 255, 255, 0.9)',
                    radialaxis=dict(showticklabels=False)), 
                    polar_angularaxis_rotation=90)
    
fig.show()

### 5. Estados Americanos e as Doses da Vacina Pfizer
Gráfico 3D que demonstra a quantidade de duas doses da vacina Pfizer durante as semanas de 2021.

In [7]:
data_frame = run_s3(BUCKET_KEY_DOSES_PFIZER)

data_frame['week_of_allocations'] = pd.to_datetime(data_frame['week_of_allocations']).dt.date

fig = px.scatter_3d(data_frame,
                    x='total_first_doses',
                    y='week_of_allocations',
                    z='total_second_doses',
                    size='total_first_doses', 
                    color='jurisdiction',    
                    hover_name='jurisdiction',   
                    labels={'total_first_doses':'Alocações da primeira dose', 'total_second_doses':'Alocações da segunda dose', 'jurisdiction':'Estado', 'week_of_allocations':'Semana'},
                    title='Distribuição de Primeiras e Segundas Doses da Vacina Pfizer')

# Melhorar a formatação da data no eixo Y para exibir apenas o mês e o ano
fig.update_layout(xaxis=dict(tickformat="%m-%Y"))
  
fig.show()

## 🏁Conclusão

Através dessa análise abrangente dos dados provenientes do AWS COVID-19 data lake, é notório a identificação de padrões críticos e correlações significativas que informam a situação passada mundialmente. 


### Principais Descobertas:

As flutuações mensais nos casos de COVID-19 e nas taxas de mortalidade revelam um padrão claro de ondas pandêmicas, que parecem estar inversamente relacionadas com as medidas de bloqueio.

As regiões com menor capacidade hospitalar e menor acesso à vacinação experienciaram taxas de mortalidade significativamente mais altas, sugerindo que a desigualdade no acesso ao cuidado de saúde e a vacinas é um fator crítico na determinação dos resultados da saúde.

### Implicações:

Esses padrões sublinham a necessidade de políticas públicas adaptadas que respondam às características demográficas e às capacidades de saúde locais. O sucesso na redução dos picos de infecção e na minimização do impacto nos serviços de saúde depende crucialmente de respostas rápidas e bem informadas às tendências emergentes da pandemia.